In [332]:
p0 = """
foo(1).
foo(2).
baz(3,1).
query(X,Y) :- foo(X), baz(Y,X).
"""

In [174]:
from pyparsing import Word, alphas, nums, Literal, Or, ParseException, ZeroOrMore, Group, delimitedList
from enum import Enum

identifier = Word(alphas)
value = Word(nums)
variable = identifier ^ value

clause = identifier.setResultsName("predicate") + "(" + delimitedList(variable, delim=",").setResultsName("variables") + ")"
rule = clause.setResultsName("head") + ":-" + delimitedList(Group(clause), delim=",").setResultsName("body")
statement = Group((clause ^ rule) + Or([".", "?"]))
program = OneOrMore(statement)

In [175]:
clause.parseString("foo(1)")
pp = program.parseString(p0)

In [117]:
dict(pp[2])["body"][0]["predicate"]

'foo'

In [346]:
from collections import defaultdict
from copy import deepcopy

def is_variable(v):
    return v[0].isalpha() and v[0].isupper()

def evaluate(statements):
    statements = [s.asDict() for s in statements]
    memory = defaultdict(lambda: [])
    while len(statements) > 0:
        p = statements[0]
        new_statements = handle(memory, p)
#         print([s["head"]["variables"] for s in new_statements])
        statements = statements[1:] + new_statements
#         print(memory, statements)
        print(statements)
    return memory
        

def find_matching_indices(l1, l2):
    r = list()
    for el in l2:
        if el in l1:
            r.append(l1.index(el))
        else:
            r.append(None)
    return r

def handle(memory, s):
    new_statements = list()
    if "head" in s:
        if "assignments" not in s["head"]:
            s["head"]["assignments"] = [None for v in s["head"]["variables"]]
        if len(s["body"]) == 0:
            memory[s["head"]["predicate"]].append(s["head"]["assignments"])
        for cidx in range(len(s["body"])):
            c = s["body"][cidx]
            matches = find_matching_indices(list(s["head"]["variables"]), list(c["variables"]))
            print(matches)
            if any(m is None for m in matches):
                continue
            for values in memory[c["predicate"]]:
                new_s = deepcopy(s)
                print("Starting head", new_s["head"])
                print("Starting body", new_s["body"])
                try:
                    for i in range(len(matches)):
    #                     if matches[i] is not None:
                        if new_s["head"]["assignments"][matches[i]] is None:
                            print(new_s["head"]["variables"][matches[i]], "->", values[i])
                            new_s["head"]["assignments"][matches[i]] = values[i]
                        else:
                            if new_s["head"]["assignments"][matches[i]] != values[i]:
                                raise Exception("Conflicting values:", new_s["head"]["assignments"][matches[i]], values[i])
                    new_s["body"] = new_s["body"][:cidx] + new_s["body"][cidx+1:]
                    print("Ending head", new_s["head"])
                    print("Ending body", new_s["body"])
                    new_statements.append(new_s)
                except Exception:
                    pass
#             print(new_statements)
    else:
        memory[s["predicate"]].append(s["variables"])
    return new_statements

            
def test_evaluate(statements):
    statements = [s.asDict() for s in statements]
    memory = defaultdict(lambda: [])
    while len(statements) > 0:
        new_statements = handle(memory, statements[0])
        statements = statements[1:] + new_statements
        print("Memory:", memory)
    return memory


In [347]:
test_evaluate(program.parseString("""
person(alice).
person(bob).
person(eve).
parent(alice,eve).
parent(bob,eve).
sibling(A, B) :- parent(A,C), parent(B,C).
"""))["sibling"]

Memory: defaultdict(<function test_evaluate.<locals>.<lambda> at 0x110a1f158>, {'person': [['alice']]})
Memory: defaultdict(<function test_evaluate.<locals>.<lambda> at 0x110a1f158>, {'person': [['alice'], ['bob']]})
Memory: defaultdict(<function test_evaluate.<locals>.<lambda> at 0x110a1f158>, {'person': [['alice'], ['bob'], ['eve']]})
Memory: defaultdict(<function test_evaluate.<locals>.<lambda> at 0x110a1f158>, {'person': [['alice'], ['bob'], ['eve']], 'parent': [['alice', 'eve']]})
Memory: defaultdict(<function test_evaluate.<locals>.<lambda> at 0x110a1f158>, {'person': [['alice'], ['bob'], ['eve']], 'parent': [['alice', 'eve'], ['bob', 'eve']]})
[0, None]
[1, None]
Memory: defaultdict(<function test_evaluate.<locals>.<lambda> at 0x110a1f158>, {'person': [['alice'], ['bob'], ['eve']], 'parent': [['alice', 'eve'], ['bob', 'eve']]})


[]

In [339]:
test_evaluate(program.parseString(p0))["query"]

defaultdict(<function test_evaluate.<locals>.<lambda> at 0x110937b70>, {'foo': [['1']]})
defaultdict(<function test_evaluate.<locals>.<lambda> at 0x110937b70>, {'foo': [['1'], ['2']]})
defaultdict(<function test_evaluate.<locals>.<lambda> at 0x110937b70>, {'foo': [['1'], ['2']], 'baz': [['3', '1']]})
[0]
X -> 1
X -> 2
[1, 0]
Y -> 3
X -> 1
defaultdict(<function test_evaluate.<locals>.<lambda> at 0x110937b70>, {'foo': [['1'], ['2']], 'baz': [['3', '1']]})
[1, 0]
Y -> 3
defaultdict(<function test_evaluate.<locals>.<lambda> at 0x110937b70>, {'foo': [['1'], ['2']], 'baz': [['3', '1']]})
[1, 0]
Y -> 3
defaultdict(<function test_evaluate.<locals>.<lambda> at 0x110937b70>, {'foo': [['1'], ['2']], 'baz': [['3', '1']]})
[0]
defaultdict(<function test_evaluate.<locals>.<lambda> at 0x110937b70>, {'foo': [['1'], ['2']], 'baz': [['3', '1']]})
defaultdict(<function test_evaluate.<locals>.<lambda> at 0x110937b70>, {'foo': [['1'], ['2']], 'baz': [['3', '1']], 'query': [['1', '3']]})
defaultdict(<functi

[['1', '3'], ['1', '3']]

defaultdict(<function test_evaluate.<locals>.<lambda> at 0x111ca2400>, {'person': [['alice']]})
defaultdict(<function test_evaluate.<locals>.<lambda> at 0x111ca2400>, {'person': [['alice'], ['bob']]})
defaultdict(<function test_evaluate.<locals>.<lambda> at 0x111ca2400>, {'person': [['alice'], ['bob'], ['eve']]})
defaultdict(<function test_evaluate.<locals>.<lambda> at 0x111ca2400>, {'person': [['alice'], ['bob'], ['eve']], 'parent': [['alice', 'eve']]})
defaultdict(<function test_evaluate.<locals>.<lambda> at 0x111ca2400>, {'person': [['alice'], ['bob'], ['eve']], 'parent': [['alice', 'eve'], ['bob', 'eve']]})
[0, None]
[1, None]
defaultdict(<function test_evaluate.<locals>.<lambda> at 0x111ca2400>, {'person': [['alice'], ['bob'], ['eve']], 'parent': [['alice', 'eve'], ['bob', 'eve']]})
[0, 1]
defaultdict(<function test_evaluate.<locals>.<lambda> at 0x111ca2400>, {'person': [['alice'], ['bob'], ['eve']], 'parent': [['alice', 'eve'], ['bob', 'eve']], 'sibling': []})


[]

In [249]:
find_matching_indices(["X", "Y"], ["X"])

[0]

In [ ]:
    
# def handle(memory, s):
#     new_statements = list()
#     if "head" in s:
#         # treat as a rule
#         if len(s["body"]) == 0:
#             memory[s["head"]["predicate"]].append(s["head"]["variables"])
#         for cidx in range(len(s["body"])):
#             c = s["body"][cidx]
#             variable_matches = find_matching_indices(list(s["head"]["variables"]), list(c["variables"]))
#             if all(m is None for m in variable_matches):
#                 continue
#             for values in memory[c["predicate"]]:
#                 print(values)
#                 new_s = dict(s)
#                 print(len(new_s["body"]))
#                 new_s["body"] = new_s["body"][:cidx-1] + new_s["body"][cidx+1:]
#                 print(len(new_s["body"]))
#                 for i in range(len(variable_matches)):
#                     if variable_matches[i] is not None:
#                         print(new_s["head"]["variables"][variable_matches[i]], "->", values[i])
#                         new_s["head"]["variables"][variable_matches[i]] = values[i]
#                 new_statements.append(new_s)
#                 print([s["head"]["variables"] for s in new_statements])
#                 print(variable_matches)
#             print(new_statements[0] is new_statements[1])
#             print(len(new_statements))
#     else:
#         memory[s["predicate"]].append(s["variables"])
#     return new_statements

In [199]:
list(range(5))[:4]

[0, 1, 2, 3]

In [256]:
a = {"foo":1}
b = dict(a)
c = dict(a)
b["foo"] = 2
print(a,b, c)
a

{'foo': 1} {'foo': 2} {'foo': 1}


{'foo': 1}

In [259]:
a is c

False

In [245]:
l = list()
l.append(None)

In [288]:
l = list(range(10))

In [289]:
l[:5] + l[6:]

[0, 1, 2, 3, 4, 6, 7, 8, 9]